# 네이버 뉴스 기사 스크래핑
>## Crawling Target
1. news_url
2. news_title
3. news_sub_title
4. news_author
5. news_date
6. news_article
7. news_img_path
8. news_source

In [17]:
%%time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


# 접근정보 추가
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15'}

# 각 기사의 URL 주소를 리스트에 담는 함수
# input 변수 = page_num: 크롤링할 페이지 수, code: 카테고리, date: 날짜
def make_urllist(page_num, code, date): 

  # URL 주소를 저장하기 위한 비어있는 리스트 미리 세팅.
  urllist= []

  # 1 ~ page_num까지 정해진 페이지만큼 반복.
  # <예시> range(1,2) = 1 페이지만 크롤링
  # range(1,11) 은 숫자 1부터 10까지 (1이상 11미만)의 숫자를 데이터로 갖는 객체
  for i in range(1, page_num + 1):

	# 입력한 변수를 조합하여 생성한 주소를  url 이라는 변수에 저장
  # 이때, 문자열과 숫자는 더할 수 없으므로 숫자를 문자열로 바꿔주어야 함
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   

	# requets 패키지의 get 함수로 호출하여 html 정보를 news라는 변수에 저장
	# import로 호출한 패키지의 함수 사용 방법은 '패키지이름.모듈이름' 따라서 requests.get 으로 사용
    news = requests.get(url, headers=headers)
    news.content

    # BeautifulSoup 모듈을 사용하여 HTML 페이지를 분석
    soup = BeautifulSoup(news.content, 'html.parser')

    # 각 페이지의 뉴스가 news_list 라는 리스트에 저장
    #html 구조 확인
      #페이지당 뉴스기사는 20개이며 상단 10개 뉴스기사는 type06_headline 클래스에 하단 10개 뉴스기사는 type06 클래스에 들어있음. 따라서 크롤링 시 두 클래스의 태그속 url을 모두 가져와야함.
    news_lists = soup.select('.type06_headline li dl')
    news_lists.extend(soup.select('.type06 li dl'))

    # 뉴스 리스트에 있는 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소' 텍스트만 가져와서 list에 append사용해 쌓기
    for news_list in news_lists:
        urllist.append(news_list.a.get('href'))
  return urllist

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 26.2 µs


In [18]:
%%time
#카테고리 뉴스 검색
news_urllist = make_urllist(1, 101, 20220220)
# make_urllist(page_num, code, date)
  # 카타고리 code
    # 정치: 100
    # 경제제: 101
    # 사회: 102
    # 생활/문화: 103
    # IT/과학: 105
    # 세계: 104
  # date: YYYYMMDD

print('뉴스 기사의 개수 :',len(news_urllist))
print(news_urllist)

뉴스 기사의 개수 : 20
['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0011016010', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=018&aid=0005150602', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0005047246', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0005047245', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=056&aid=0011216462', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=056&aid=0011216461', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=277&aid=0005047244', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0011015948', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000787128', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=660&aid=0000002275', 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=052&

In [16]:
%%time
data = []
for news_url in news_urllist:
  json_data = {}
  
  # news_url: 기사 원문 URL
  json_data["news_url"] = news_url
  print(news_url)

  news_url = requests.get(news_url, headers=headers)
  news_url.content
  news_page_html = BeautifulSoup( news_url.content, 'html.parser')

  # news_title: 기사 제목
  news_title = news_page_html.select_one('#articleTitle').text
  json_data["news_title"] = news_title
  print(news_title)

  # news_subtitle: 부제목
  if news_page_html.select_one(".media_end_summary"):
    news_subtitle = news_page_html.select_one(".media_end_summary").text
  else:
    news_subtitle = "null"
  json_data["news_subtitle"] = news_subtitle
  print(news_subtitle)

  # news_author: 기자
  if news_page_html.select_one(".journalistcard_summary_name"):
    news_author = news_page_html.select_one(".journalistcard_summary_name").text
    news_author = re.sub("[기자]","", news_author).strip()
  elif news_page_html.select_one(".b_text"):
    news_author = news_page_html.select_one(".b_text").text
    news_author = re.sub("[|기자|인턴|특파원|\s|\d|a-z|(|@|.|)|]","", news_author).strip()
  else:
    news_author = "null"
  json_data["news_author"] = news_author
  print(news_author)

  # news_date: 날짜
  if news_page_html.select_one(".t11"):
    news_data = news_page_html.select_one(".t11").text
  #   # news_date = str(news_data)
  #   news_date = news_date.replace('<span class="t11">', "")
  #   news_date = news_date.replace("</span>", "")
  else:
    news_data = "null"
  json_data["news_data"] = news_data
  print(news_data)

  # news_article: 기사 내용
  if news_page_html.find('div', {'id':'articleBodyContents'}):
    news_article = news_page_html.find('div', {'id':'articleBodyContents'}).text
    news_article = news_article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
    news_article = news_article.replace("function _flash_removeCallback() {}", "").strip()
    ignore_article = "|\n|\t|\xa0|[WEEKLY|BIZ]|Biz|Calendar|[앵커]|[리포트]|이미지출처처=연합뉴스스]|[KBS|울산]|[포토]|[연합뉴스]|[경향신문]|"
    news_article = re.sub(ignore_article,"", news_article)
  else:
    news_article = "null"
  json_data["news_article"] = news_article
  print(news_article)

  # news_img_path: 기사 img 경로
  # if news_soup.select("figure"):
  #     news_img_path = news_soup.select_one("figure").img.get("src")
  # else:
  #     news_img_path = "null"
  # json_data["news_img_path"] = news_img_path

  # news_source : 신문사
  if news_page_html.select_one(".c_text"):
    news_source = news_page_html.select_one(".c_text").get_text().strip()
    news_source = news_source.replace("Copyright ⓒ ", "")
    news_source = news_source.replace(". All rights reserved. 무단 전재 및 재배포 금지.", "")
  else:
    news_source = "null"
  json_data["news_source"] = news_source
  print(news_source)

  data.append(json_data)

print(data)
  



https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0011016010
홍콩 코로나 6067명 사흘째 6천명대 ‘폭증’·총 5만2830명...14명 추가 사망
null
이재준
2022.02.20. 오후 11:59
[홍콩AP/시]18일(현시간)홍콩코즈웨베쇼핑가의한빈식당에서곳직원손님을기다고있다.홍콩의코로나19환자가4천명을돌파하는등폭증세를보자당국은750만전주민에코로나19강제검사를하기로했다.에따라홍콩시민은다음달초부터일주일에1회씩,3주동안총3회의검사를받아야하며검사를거부할우약153만원의벌금부과된다.2022.02.18[서시]재준기자홍콩에서코로나19환자가오크론변를중심으로크게확하는가운데사흘속6000명상발병했다.동망(東網)등에따르면위생방호센터촹숙콴(張竹君)전염병주임은20일규확진자가6067명발병해누적감염자가5만2830명에달했다고밝혔다.촹숙콴주임은추가감염자가운데6055명홍콩에서역감염했고나머12명은국외에서유입했다고전했다.추가확진자가운데3188명오크론변,34명은델타변에각각걸렸으며114명정밀검사결과를기다고있다.의원관국은재차14명의환자가사망했다고집계했다.남성7명과여성7명으로나는70세에서94세에르렀다.잠정적으로총290명에달했다.공립병원입원자중10여명위독한상황고수십명은위중한상황어서숨는환자가더욱늘어날전망다.또한의원관국은난24시간동안확진자227명완치퇴원했만격병상사용률90%를넘어섰고집중치료실우사용률60%에달했으며입원자대부분고령환자라고소개했다.홍콩에서는3월초주민750만명을대상으로3차례전수검사를실시할계획다.1주일마다3주일속검사를마쳐감염자를모두찾아내격시켜도시봉쇄를피하도록할방침다.아러홍콩정부는코로나19확을염려해행정장관선거를기하기로했다.캐람(林鄭月娥)홍콩행정장관은내달27일로예정한차기행정장관선거는5월8일로기한다고발표했다.
뉴시스
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=018&aid=0005150602
[클릭, 글로벌 제약·바이오] 포스트코로

In [ ]:
len(data)

20

In [ ]:
# json 파일로 저장
import json
with open("Naver_News.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file,  ensure_ascii=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
